In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB4 
from keras_tuner import RandomSearch

# 데이터셋 경로 설정
train_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 7775 images belonging to 8 classes.
Found 1670 images belonging to 8 classes.
Found 1667 images belonging to 8 classes.


In [2]:
def build_model(hp):
    base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    
    for layer in base_model.layers:
        layer.trainable = False

    for layer in base_model.layers[-20:]:
        layer.trainable = True   
    
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    # Dense 레이어의 유닛 수를 하이퍼파라미터로 사용
    model.add(layers.Dense(units=hp.Int('units', min_value=256, max_value=512, step=128), activation = 'relu'))
    model.add(layers.Dense(8, activation = 'softmax'))

    # learning_rate를 하이퍼파라미터로 사용
    model.compile(optimizer=optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-3, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])    
    
    return model


In [3]:
import keras_tuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 시도할 하이퍼파라미터 조합의 최대 개수
    executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
    directory=r'E:\AInotes\자세교정\모델학습\EffiNet\random_search',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
    project_name='EN_BC_RS_01'  # 프로젝트 이름
)
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 256, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0, verbose=1)

tuner.search(train_generator,
             steps_per_epoch=train_generator.samples // train_generator.batch_size,
             validation_data=validation_generator, 
             validation_steps=validation_generator.samples // validation_generator.batch_size,
             epochs=25,
             callbacks=[early_stopping])


# 최적의 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# 최적의 하이퍼파라미터로 모델 빌드
model = tuner.hypermodel.build(best_hps)

#best_models = tuner.get_best_models(1)
#final_model = best_models[0]

Trial 5 Complete [00h 03m 59s]
val_accuracy: 0.1328125

Best val_accuracy So Far: 0.13701923191547394
Total elapsed time: 00h 24m 12s


In [5]:
# 모든 최적 하이퍼파라미터 출력
for hp in best_hps.values:
    print(f"The optimal value for {hp} is {best_hps.get(hp)}")

# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=40,
                    callbacks=[early_stopping])

The optimal value for units is 512
The optimal value for learning_rate is 0.00010367752771388617
Epoch 1/40
242/242 [==============================] - 25s 74ms/step - loss: 2.1047 - accuracy: 0.1248 - val_loss: 2.0816 - val_accuracy: 0.1322
Epoch 2/40
242/242 [==============================] - 15s 63ms/step - loss: 2.0878 - accuracy: 0.1308 - val_loss: 2.0806 - val_accuracy: 0.1316
Epoch 3/40
242/242 [==============================] - 15s 62ms/step - loss: 2.0850 - accuracy: 0.1239 - val_loss: 2.0802 - val_accuracy: 0.1256
Epoch 4/40
242/242 [==============================] - 15s 61ms/step - loss: 2.0812 - accuracy: 0.1231 - val_loss: 2.0819 - val_accuracy: 0.1226
Epoch 5/40
242/242 [==============================] - 15s 61ms/step - loss: 2.0820 - accuracy: 0.1179 - val_loss: 2.0794 - val_accuracy: 0.1286
Epoch 6/40
242/242 [==============================] - 15s 61ms/step - loss: 2.0808 - accuracy: 0.1263 - val_loss: 2.0795 - val_accuracy: 0.1382
Epoch 7/40
242/242 [===================